In [11]:
from flask import Flask
from flask import request
from flask import jsonify
import pandas as pd
import requests
import json
import urllib.request

app = Flask(__name__)



In [18]:
apikey = "aaf421cd39a23adebb77ad9c18e7e438"
url = f"https://gnews.io/api/v4/search?q=liberdade&lang=pt&country=br&max=10&apikey={apikey}"

with urllib.request.urlopen(url) as response:
    data = json.loads(response.read().decode("utf-8"))
    articles = data["articles"]

    for i in range(len(articles)):
        # articles[i].title
        print(f"Title: {articles[i]['title']}")
        # articles[i].description
        print(f"Description: {articles[i]['description']}")
        # You can replace {property} below with any of the article properties returned by the API.
        # articles[i].{property}
        # print(f"{articles[i]['{property}']}")

        # Delete this line to display all the articles returned by the request. Currently only the first article is displayed.
        break

Title: Economist fala em 'nova guerra fria' e aponta Taiwan como 'ponto de ignição'
Description: Revista britânica sugere que os taiwanêses estejam "prontos para fazer sacrifícios" por sua "liberdade"


In [36]:

url_sa = "https://api.meaningcloud.com/sentiment-2.1"
key_sa = "76dba976cdc2a85a79b3452591793cd0"

def get_sentiment(text):
    payload = {
        'key': key_sa,
        'txt': text,
        'lang': 'pt'  # definir o idioma do texto aqui
    }
    response = requests.post(url_sa, data=payload)
    if response.status_code == 200:
        result = response.json()
        return result['score_tag']
    else:
        return None

In [32]:
df = pd.DataFrame(data=articles)
df = df[['title', 'description']]

In [34]:
df['news'] = df.apply(lambda row: f'Título: {row["title"]} \n Descrição: {row["description"]}', axis=1)


In [37]:
df['sentiment'] = df['news'].apply(get_sentiment)
df['sentiment'] = df['sentiment'].apply(lambda x: 'NEUTRO' if x == 'NEU' else 'POSITIVO' if x == 'P' else 'NEGATIVO' if x == 'N' else 'Outros')

In [43]:
df_final = df[['news', 'sentiment']]
df_final.to_csv('g1_news.csv')

Código Antigo (Funcionando, porém, sem tratamento de erros)

In [ ]:
'''from flask import Flask
from flask import request
from flask import jsonify
from config.apikey import apikey
from functions import get_sentiment
import pandas as pd
import requests
import json
import urllib.request

app = Flask(__name__)


# Configurando a api de coleta de dados
url = f"https://gnews.io/api/v4/search?q=liberdade&lang=pt&country=br&max=10&apikey={apikey}"

with urllib.request.urlopen(url) as response:
    data = json.loads(response.read().decode("utf-8"))
    articles = data["articles"]

# lê o arquivo csv
news = pd.read_csv('g1_news.csv')

# endpoint que retorna todos os dados do dataframe
@app.route('/data', methods=['GET'])
def get_data():
    return jsonify(news.to_dict(orient='records'))

# endpoint que insere novos dados no dataframe
@app.route('/data', methods=['POST'])
def add_data():
    data = request.json
    news = news.append(data, ignore_index=True)
    news.to_csv('g1_news.csv', index=False)
    return jsonify({"message": "Dados adicionados com sucesso."}), 201

# endpoint que retorna um registro específico do dataframe
@app.route('/data/<id>', methods=['GET'])
def get_data_by_id(id):
    data = news.loc[news['id'] == id]
    if data.empty:
        return jsonify({"message": "Registro não encontrado."}), 404
    return jsonify(data.to_dict(orient='records'))

# endpoint que atualiza um registro específico do dataframe
@app.route('/data/<id>', methods=['PUT'])
def update_data(id):
    data = request.json
    updated_data = news.loc[news['id'] == id]
    if updated_data.empty:
        return jsonify({"message": "Registro não encontrado."}), 404
    updated_data.update(data)
    news.to_csv('g1_news.csv', index=False)
    return jsonify({"message": "Dados atualizados com sucesso."}), 200

# endpoint que exclui um registro específico do dataframe
@app.route('/data/<id>', methods=['DELETE'])
def delete_data(id):
    deleted_data = news.loc[news['id'] == id]
    if deleted_data.empty:
        return jsonify({"message": "Registro não encontrado."}), 404
    news.drop(news[news['id'] == id].index, inplace=True)
    news.to_csv('g1_news.csv', index=False)
    return jsonify({"message": "Registro excluído com sucesso."}), 200

    
if __name__ == '__main__':
    app.run()
'''